In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import fitparse as fp
import csv
import pytz
import datetime
from pathlib import Path
import os
import shutil
from pathlib import Path
import re

In [2]:
def print_record(data):
    """
    test method to print out the data in a record
    """
    for record in data:
        for field in record:
            if field.units:
                print (" * %s: %s %s" % (
                    field.name, field.value, field.units))
            else:
                print (" * %s: %s" % (field.name, field.value))

In [3]:
def get_time(record):
    """
    extract the time from data. Note for heart rate data the timestamp is int(16) up to 65536 ~ 18hours
    This will overflow and needs normalised so it wont give a crazy result on overflow.

    Every now and then there will be a timestamp giving the right date and time. 
    Heart rate data is timestamped on a rolling seconds counter. Need to set the first timestamp_16 to the
    global time then work out the next time based on the difference between timestamp_16s adding that to global time
    """
    new_timestamp_16 = None
    
    # difference between successive timestamp_16
    delta_diff = 0
    global timestamp_16
    global current_timestamp

    for r in record:
        if r.field is not None:
            if r.name == 'timestamp_16':
                new_timestamp_16 = r.value

    # if the timestamp is none then set it to the global time
    if new_timestamp_16 is not None:
        if timestamp_16 is None:
            timestamp_16 = new_timestamp_16
        
        delta_diff = new_timestamp_16 - timestamp_16
        
        # deal with overflow
        if delta_diff < 0:
            delta_diff += 65536
        
        current_timestamp += datetime.timedelta(seconds=delta_diff)
        timestamp_16 = new_timestamp_16
        #print(current_timestamp)
        return current_timestamp

In [4]:
def get_heartrate(record):
    """
    Return the heart rate from the record
    """
    for r in record:
        if r.name == 'heart_rate':
            return r.value

In [5]:
def output_messages(fitfile):
    """
    Go through the fitfile and extract time and heart rate data
    """
    messages = fitfile.get_messages()

    current_timestamp = datetime.datetime.now()
    timestamp_16 = None

    hrdata = []
    timestamp = []

    for record in messages:
    # Go through all the data entries in this record

        # Extract the global time stamp
        if record.name == 'monitoring_info':
            for r in record.fields:
                if r.field is not None:
                    if r.field.name == 'local_timestamp':
                        current_timestamp = r.value
                        timestamp_16 = None
                        #print("Timestamp: {}".format(current_timestamp))

        # get the heart rate data
        if record.name == 'monitoring':
            for r in record:
                if r.field is not None:
                    if r.field.name =='heart_rate':
                        time = get_time(record)
                        hr = get_heartrate(record)

                        timestamp.append(time)
                        hrdata.append(hr)
                        #print("Time: {}   Heart: {}".format(time,hr))

In [6]:
def output_csv(time, hr, fi):
    """
    Write the extracted time and heart rate data to csv fiile
    """
    data = [time, hr]

    csvFile = open(fi,'w')
    with csvFile:
        print("Writing csv to: {}".format(fi))
        writer = csv.writer(csvFile, lineterminator='\n') # need lineterminator to prevent extra linebreaks
        writer.writerow(["Date time", "Heart rate BPM"])
        for i in range(len(time)):
            writer.writerow([time[i],hr[i]])

In [7]:

def copy_files_without_email(source, destination):
    r = []
    for root, dirs, files in os.walk(source):
        for name in files:
            filepath = root + os.sep + name
            if filepath.endswith(".fit"):
                name = Path(filepath)
                #print(name.stem)
                no_email = re.findall(r'(?<=_).*$', name.stem) # look for anything after _
                #print(no_email[0])
                if len(no_email)  is not None:
                    new_name = no_email[0] + name.suffix
                    dest = destination + new_name
                    #print(dest)
                    shutil.copy(filepath, destination+new_name)
                

In [8]:
raw_data_dir = Path(r"Garmin_data/data")
dest_data_dir = Path(r"Garmin_data/fit/")

In [9]:
#copy_files_without_email(raw_data_dir, dest_data_dir)

In [10]:
#os.listdir(dest_data_dir)

In [11]:
#grab last 4 characters of the file name:
def filenumber_beginning(x):
    # get file name (xxx.fit) and split on .fit
    # if filename has xxx_yyy.fit then split on _yyy.fit
    # return xxx
    filename = os.path.splitext(x)[0]
    
    if '_' in filename:
        filename = filename.split('_')[0]

    file_out = int(filename)

    return(file_out)

file_list = os.listdir(dest_data_dir)

s = sorted(file_list, key = filenumber_beginning) 

print(f"files {len(s)}")

files 8523


In [30]:
parse_file = dest_data_dir.joinpath(s[100])
fitfile = fp.FitFile(str(parse_file)) # needs string file path and not windows path that sorted produces

In [33]:
xt310 = "fr310xt_4t"
xt310_no = "1080"
xt735 = "2158"
watch_dev_no = [xt310, xt310_no, xt735]

In [42]:
message_dict = {
"file_id" : "file_id",
"record" : "record",
"device_info" : "device_info"
}

In [49]:
di = fitfile.get_messages(message_dict["file_id"])
product = []
for r in di:
    print(f"-{r}")
    product = list(filter( lambda obj: obj.name=="garmin_product", r))

print(product[0].value)

-file_id (#0)
fr310xt_4t


for 310_xt it is all records. 
Pull out `timestamp: 2017-01-18 19:31:28` and `heart_rate: 160 [bpm]`

In [52]:
if product[0].value is xt310_no or xt310_no:
    print(product[0].value)

fr310xt_4t


In [92]:
recs = fitfile.get_messages(message_dict["record"])
heartrate = []
timestamp = []

for record in recs:
    hr = None
    time = None
    hr = list(filter(lambda a: a.name == 'heart_rate', record))
    time =  list(filter(lambda a: a.name == 'timestamp', record))

    if hr is not None and time is not None:    
        heartrate.append(hr)
        timestamp.append(time)

for h in heartrate:
    print(h[0].value)

for t in timestamp:
    print(t[0].value)
    print(type(t[0].value))

546
546
69
69
69
73
74
76
79
79
80
79
79
81
85
87
88
92
97
97
99
99
101
105
106
106
110
111
114
115
118
118
118
120
121
122
122
122
122
122
122
122
122
120
120
121
122
122
125
125
129
129
128
129
129
129
129
129
129
130
131
131
133
133
133
134
134
134
136
136
139
141
143
143
144
144
143
144
146
147
146
146
146
147
147
147
149
147
147
149
148
149
150
150
150
150
149
148
149
149
149
150
149
149
149
145
144
141
139
137
134
134
132
131
130
129
129
132
133
136
136
138
139
139
142
143
145
145
145
146
147
147
148
150
152
154
155
156
155
154
155
155
154
153
152
152
152
150
150
149
149
148
147
146
145
146
148
150
150
152
154
157
157
158
159
159
160
161
162
162
162
160
158
158
157
157
158
157
156
155
155
155
157
157
156
156
155
153
152
152
153
153
153
153
152
152
151
133
131
128
128
128
128
129
132
132
134
135
136
136
137
139
140
140
140
141
142
144
144
144
145
147
147
148
148
146
145
142
142
140
139
137
137
137
135
135
134
133
133
133
131
132
132
132
131
130
130
132
133
134
135
134
132
131
132


In [45]:
di = fitfile.get_messages(message_dict["file_id"])
print(str(di))
for r in di:
    print(f"-{r}")
    for x in r:
        print(f"    {x}")

<generator object FitFile.get_messages at 0x000001D0EEADC640>
-file_id (#0)
    garmin_product: fr310xt_4t
    manufacturer: garmin
    number: None
    serial_number: 3871167223
    time_created: 2017-01-18 18:49:35
    type: activity


In [34]:
di = fitfile.get_messages()
for r in di:
    print(f"-{r}")
    for x in r:
        print(f"    {x}")

-file_id (#0)
    garmin_product: fr310xt_4t
    manufacturer: garmin
    number: None
    serial_number: 3871167223
    time_created: 2017-01-18 18:49:35
    type: activity
-file_creator (#49)
    hardware_version: None
    software_version: 450
-event (#21)
    event: timer
    event_group: 0
    event_type: start
    timer_trigger: manual
    timestamp: 2017-01-18 18:49:34
-device_info (#23)
    battery_status: None
    battery_voltage: None [V]
    cum_operating_time: None [s]
    device_index: creator
    device_type: 1
    garmin_product: fr310xt_4t
    hardware_version: None
    manufacturer: garmin
    serial_number: 3871167223
    software_version: 4.5
    timestamp: 2017-01-18 18:49:34
    unknown_15: None
    unknown_16: None
    unknown_8: None
    unknown_9: None
-device_info (#23)
    battery_status: None
    battery_voltage: None [V]
    cum_operating_time: None [s]
    device_index: 1
    device_type: 12
    garmin_product: 1080
    hardware_version: None
    manufactur

In [35]:
di = fitfile.get_messages('record')
for r in di:
    print(f"-{r}")
    for x in r:
        print(f"    {x}")

-record (#20)
    altitude: 115.0 [m]
    distance: 1.05 [m]
    enhanced_altitude: 115.0 [m]
    enhanced_speed: 0.993 [m/s]
    heart_rate: 69 [bpm]
    position_lat: 636588994 [semicircles]
    position_long: -15625416 [semicircles]
    speed: 0.993 [m/s]
    timestamp: 2017-01-18 18:49:34
-record (#20)
    altitude: 115.0 [m]
    distance: 1.64 [m]
    enhanced_altitude: 115.0 [m]
    enhanced_speed: 1.054 [m/s]
    heart_rate: 69 [bpm]
    position_lat: 636589060 [semicircles]
    position_long: -15625499 [semicircles]
    speed: 1.054 [m/s]
    timestamp: 2017-01-18 18:49:35
-record (#20)
    altitude: 115.39999999999998 [m]
    distance: 12.41 [m]
    enhanced_altitude: 115.39999999999998 [m]
    enhanced_speed: 1.934 [m/s]
    heart_rate: 69 [bpm]
    position_lat: 636589692 [semicircles]
    position_long: -15627107 [semicircles]
    speed: 1.934 [m/s]
    timestamp: 2017-01-18 18:49:38
-record (#20)
    altitude: 115.60000000000002 [m]
    distance: 18.57 [m]
    enhanced_alt

In [26]:
di = fitfile.get_messages('device_info')
for r in di:
    for x in r.fields:
        if x.name =="garmin_product":
            print(x.value)

2158


In [14]:
# for f in os.listdir(dest_data_dir):

#     filename = os.path.splitext(f)[0]
    
#     if '_' in filename:
#         filename = filename.split('_')[0]

#     filename = int(filename)
#     print(filename)

In [15]:
# # get the fit files in order (maybe)
# fit_files_sorted = sorted(Path(dest_data_dir).glob('*.fit'))

In [16]:
# fit_files_sorted[0].__str__()

In [17]:
# fitfile = fp.FitFile(fit_files_sorted[-1].__str__()) # needs string file path and not windows path that sorted produces
# fitfile

In [18]:
mesx = fitfile.get_messages()
mesx

<generator object FitFile.get_messages at 0x000001D0EC8D2740>

In [19]:
for r in mesx:
    print(f"-{r}")
    for x in r:
        print(f"    {x}")

-file_id (#0)
    garmin_product: fr310xt_4t
    manufacturer: garmin
    number: None
    serial_number: 3871167223
    time_created: 2017-01-18 18:49:35
    type: activity
-file_creator (#49)
    hardware_version: None
    software_version: 450
-event (#21)
    event: timer
    event_group: 0
    event_type: start
    timer_trigger: manual
    timestamp: 2017-01-18 18:49:34
-device_info (#23)
    battery_status: None
    battery_voltage: None [V]
    cum_operating_time: None [s]
    device_index: creator
    device_type: 1
    garmin_product: fr310xt_4t
    hardware_version: None
    manufacturer: garmin
    serial_number: 3871167223
    software_version: 4.5
    timestamp: 2017-01-18 18:49:34
    unknown_15: None
    unknown_16: None
    unknown_8: None
    unknown_9: None
-device_info (#23)
    battery_status: None
    battery_voltage: None [V]
    cum_operating_time: None [s]
    device_index: 1
    device_type: 12
    garmin_product: 1080
    hardware_version: None
    manufactur

look for device_info-> garmin_product.
The fit files will change depending on what device it is 

In [20]:
for r in mesx:
    print(f"-{r.name}")
    #for x in r:
        #print(f"    {x}")

In [21]:
for f in fit_files_sorted:
    try:
        print(f"converting {f.name}")
        fitfile = fp.FitFile(f.__str__())
        
        output_messages(fitfile)
        
    except fp.utils.FitCRCError as err:
        print(type(err))    # the exception type
        print(err.args)     # arguments stored in .args
        print(err) 
        print(f"error with {f.name}")
        
        
#fi = datadir+'/'+'op.csv'
#output_csv(timestamp, hrdata, fi)

NameError: name 'fit_files_sorted' is not defined

In [ ]:
#choose the right directory here fit files are located
dir = "data/Fit_data/"
testdir = "C:/temp/data/test"
datadir = dir

UTC = pytz.UTC
GMT = pytz.timezone('Europe/London')

# heart rate data list
hrdata = []
# corresponding timestamp
timestamp = []

# globals to get the time data
current_timestamp = datetime.datetime.now()
timestamp_16 = None

In [ ]:
# get the fit files in order (maybe)
files = sorted(Path(datadir).glob('*.fit'))

for f in files:

    ff = datadir+'/'+f.

    fitfile = fitparse.FitFile(ff)

    print("converting {}".format(f.name))

    output_messages(fitfile)

fi = datadir+'/'+'op.csv'
output_csv(timestamp, hrdata, fi)


In [22]:
np.uint16(np.iinfo(np.uint16).max) # unit16 max

65535